In [2]:
import sys
sys.path.insert(0, 'E:\pycharm\PepBenchmark\src')
sys.path.append('E:\pycharm\PepBenchmark\src\pepbenchmark\external')

import warnings
warnings.filterwarnings("ignore")

In [20]:
import dis
from pepbenchmark.single_pred.base_dataset import SingleTaskDatasetManager
from pepbenchmark.utils.logging import disable_logging,enable_logging
# disable_logging()
enable_logging()
dataset = SingleTaskDatasetManager(
    dataset_name="test",
    official_feature_names=["fasta","label"],
    force_download=True,
)
dataset.set_official_feature("fasta")
dataset.set_official_feature("smiles")
print(f"Dataset length: {len(dataset)}")
print("Official features:", dataset.official_feature_dict.keys())
dataset.set_user_feature("other_feature", list(range(len(dataset))))
print("User features:", dataset.user_feature_dict.keys())

# Set split indices
dataset.set_official_split_indices(split_type="random_split", fold_seed=1)
print("Split indices:", dataset.get_split_indices())

# Get train, valid, test features
train_features, valid_features, test_features = dataset.get_train_val_test_features(format="pytorch_dataset")
print("Train features:", train_features[0])

[2025-06-29 14:32:01,724][INFO][pepbenchmark] Feature file ====test/fasta.csv=== exists but force_download is True, will re-download
[2025-06-29 14:32:01,726][INFO][pepbenchmark] Downloading ===test/fasta.csv=== from https://raw.githubusercontent.com/ZGCI-AI4S-Pep/peptide_data/main/test/fasta.csv
[2025-06-29 14:32:02,143][INFO][pepbenchmark] Set official feature: fasta successfully
[2025-06-29 14:32:02,145][INFO][pepbenchmark] Feature file ====test/label.csv=== exists but force_download is True, will re-download
[2025-06-29 14:32:02,146][INFO][pepbenchmark] Downloading ===test/label.csv=== from https://raw.githubusercontent.com/ZGCI-AI4S-Pep/peptide_data/main/test/label.csv
[2025-06-29 14:32:42,533][ERROR][pepbenchmark] Failed to download label: ('Connection aborted.', ConnectionResetError(10054, '远程主机强迫关闭了一个现有的连接。', None, 10054, None))


ConnectionError: ('Connection aborted.', ConnectionResetError(10054, '远程主机强迫关闭了一个现有的连接。', None, 10054, None))

## Featurization

In [4]:
from pepbenchmark.pep_utils.featurizer import PeptideFeaturizer
fp_featurizer = PeptideFeaturizer("fasta", "embedding")
fp_featurizer = PeptideFeaturizer("smiles", "fingerprint", {"fp_type": "Morgan", "radius": 2})
result = fp_featurizer("CC(=O)OC1=CC=CC=C1C(=O)O")
print(result)


[2025-06-29 15:48:11,572][INFO][pepbenchmark] Failed to instantiate Fasta2Embedding. Reason: Fasta2Embedding.__init__() missing 1 required positional argument: 'model'
[2025-06-29 15:48:11,573][INFO][pepbenchmark] Expected parameters (from docstring):
[2025-06-29 15:48:11,574][INFO][pepbenchmark] 
    Convert FASTA sequence to molecular embedding using a pretrained model.
    If `model` is a string, initialize with Transformers; if it is a PyTorch model instance,
    it can be any `torch.nn.Module`. In that case, the model should provide a `tokenizer` attribute
    or have `config.name_or_path` for inference.

    Args:
        model: HuggingFace model identifier (str) or a PyTorch `nn.Module` instance.
        device: Optional device string (e.g., 'cuda', 'cpu'). Defaults to GPU if available else CPU.
    
[2025-06-29 15:48:11,575][INFO][pepbenchmark] Successfully instantiated Smiles2FP with parameters: {'fp_type': 'Morgan', 'radius': 2}


[0 0 0 ... 0 0 0]


[15:48:11] DEPRECATION WARNING: please use MorganGenerator


## User defined features and split indices

In [3]:
import pandas as pd
from pepbenchmark.single_pred.base_dataset import SingleTaskDatasetManager
from pepbenchmark.utils.logging import disable_logging,enable_logging
from pepbenchmark.pep_utils.convert import Fasta2Helm, Fasta2Biln
from pepbenchmark.pep_utils.featurizer import PeptideFeaturizer
from pepbenchmark.splitter.random_spliter import RandomSplitter
import warnings
warnings.filterwarnings("ignore")

enable_logging()
dataset = SingleTaskDatasetManager(
    dataset_name="QS_APML",
    official_feature_names=["fasta","label"],
    dataset_dir = "E:\pycharm\peptide_data\Theraputic-Other\QS_APML")
fasta = dataset.get_official_feature("fasta")
helm_converter = PeptideFeaturizer("fasta", "helm")
smi_converter = PeptideFeaturizer("fasta", "smiles")
smiles = smi_converter(fasta)
fp_featurizer = PeptideFeaturizer("smiles", "fingerprint", {"fp_type": "Morgan", "radius": 2})
helm = helm_converter(fasta)
fp = fp_featurizer(smiles)

dataset.set_user_feature("helm", helm)
dataset.set_user_feature("fingerprint", fp)

spliter = RandomSplitter()
split_indices = spliter.get_split_indices(fasta,
                                        n_splits=5,
                                        frac_train=0.8,
                                        frac_valid=0.1,
                                        frac_test=0.1,
                                        seed=42,
                                    )

dataset.set_user_split_indices(split_indices)

# Get train, valid, test features
train_features, valid_features, test_features = dataset.get_train_val_test_features(format="dict")
train_df = pd.DataFrame(train_features)
train_df.head()

INFO:pepbenchmark.metadata:Dataset directory: C:\Users\wangr/.pepbenchmark_cache/data_share/peptide_dataset/processed_2025.6.12v/
INFO:rdkit:Enabling RDKit 2025.03.3 jupyter extensions
[2025-06-29 16:31:46,318][INFO][pepbenchmark] Set official feature: fasta successfully
[2025-06-29 16:31:46,320][INFO][pepbenchmark] Set official feature: label successfully
[2025-06-29 16:31:46,321][INFO][pepbenchmark] Feature fasta already loaded, skipping download
[2025-06-29 16:31:46,322][INFO][pepbenchmark] Successfully instantiated Fasta2Helm with parameters: {}
[2025-06-29 16:31:46,322][INFO][pepbenchmark] Successfully instantiated Fasta2Smiles with parameters: {}
[2025-06-29 16:31:46,593][INFO][pepbenchmark] Successfully instantiated Smiles2FP with parameters: {'fp_type': 'Morgan', 'radius': 2}
[16:31:46] DEPRECATION WARNING: please use MorganGenerator
[16:31:46] DEPRECATION WARNING: please use MorganGenerator
[16:31:46] DEPRECATION WARNING: please use MorganGenerator
[16:31:46] DEPRECATION WARNI

,official_fasta,official_label,user_helm,user_fingerprint
395,EMRLSKFFRDFILQRKK,1,PEPTIDE1{E.M.R.L.S.K.F.F.R.D.F.I.L.Q.R.K.K}$$$...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ..."
78,DILIIVGG,1,PEPTIDE1{D.I.L.I.I.V.G.G}$$$V2.0,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
278,VEYHFLSPYVSPRE,0,PEPTIDE1{V.E.Y.H.F.L.S.P.Y.V.S.P.R.E}$$$V2.0,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
305,APTLWARMI,0,PEPTIDE1{A.P.T.L.W.A.R.M.I}$$$V2.0,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ..."
325,EIRQTHNIFFNFFKRR,1,PEPTIDE1{E.I.R.Q.T.H.N.I.F.F.N.F.F.K.R.R}$$$V2.0,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [4]:
from pepbenchmark.splitter.homo_spliter import MMseqs2Spliter
spliter = MMseqs2Spliter()
split_indices = spliter.get_split_indices(fasta,
                                        n_splits=5,
                                        frac_train=0.8,
                                        frac_valid=0.1,
                                        frac_test=0.1,
                                        identity=0.25,
                                        seed=42,
                                    )

[2025-06-29 16:31:54,305][INFO][pepbenchmark] Total clusters: 380
Cluster size distribution:
  size = 1: 365 clusters
  size 2–4: 11 clusters
  size 5–9: 2 clusters
  size 10–19: 1 clusters
  size 20+: 1 clusters

[2025-06-29 16:31:54,396][INFO][pepbenchmark] Finish clustering and splitting data.
                    
Target train data size: 348, Train: 348
                    
Target valid data size：43， Valid: 43
                    
Target test data size：43, Test: 45
